## Example analysis on CRN for cholesterol regulation in Retina

### Include relevant functions and files

In [1]:
using CRNAnalysis
using Catalyst
using Oscar
retina_rn = include("../data/retina.jl");

### Run the decomposition and generate the subnetworks

In [2]:
subnets = generate_subnets(retina_rn);

Number of independent subnetworks: 27
Subnetwork 1: [1, 4, 7]
Subnetwork 2: [2, 5, 8]
Subnetwork 3: [3, 6, 9]
Subnetwork 4: [10, 22, 24, 27, 29]
Subnetwork 5: [11, 25, 30]
Subnetwork 6: [12, 23, 26, 28]
Subnetwork 7: [13, 35, 39]
Subnetwork 8: [14, 36, 41]
Subnetwork 9: [15, 63]
Subnetwork 10: [16]
Subnetwork 11: [17, 64]
Subnetwork 12: [18, 40]
Subnetwork 13: [19, 42]
Subnetwork 14: [20, 37]
Subnetwork 15: [21, 38]
Subnetwork 16: [31]
Subnetwork 17: [32]
Subnetwork 18: [33]
Subnetwork 19: [34]
Subnetwork 20: [43]
Subnetwork 21: [44]
Subnetwork 22: [45]
Subnetwork 23: [46, 47, 49, 57, 58, 60, 61, 65, 69, 70, 48, 50, 66, 67]
Subnetwork 24: [51, 52]
Subnetwork 25: [53, 54]
Subnetwork 26: [55, 56]
Subnetwork 27: [59, 62, 68]


In [ ]:
save_subnet.(subnets[[1, 2, 3, 23]], path="retina/")

4-element Vector{String}:
 "retina/Retina_SN1.jl"
 "retina/Retina_SN2.jl"
 "retina/Retina_SN3.jl"
 "retina/Retina_SN23.jl"

### Preliminary subnetwork anlalysis

In [10]:
retina_sn1 = include("retina/Retina_SN1.jl")
@assert retina_sn1 == subnets[1]
retina_sn1

Model Retina_SN1:
Unknowns (3): see unknowns(Retina_SN1)
  Sci(t)
  Ce(t)
  C(t)
Parameters (3): see parameters(Retina_SN1)
  μ
  θ
  η

In [5]:
deficiency(retina_sn1)

1

### Catalyst can identify RPA-capable species in a deficiency 1 network

In [6]:
species(retina_sn1)[Catalyst.robustspecies(retina_sn1)]

1-element Vector{SymbolicUtils.BasicSymbolic{Real}}:
 Ce(t)

### Find the coefficients for a rowspan polynomial for subnetwork 1 involving the first and third complexes

In [7]:
b, invariant = rowspan_invariants(retina_sn1, [1, 3])
b

3×1 Matrix{Num}:
  -(1 / θ)
         0
 -(-1 / θ)

In [8]:
invariant

1×5 Matrix{Num}:
 (-μ) / θ  0  1  0  0

### Multiply the polynomial coefficients by the mass action vector to get the RPA equation for subnetwork 1

In [9]:
invariant * massactionvector(retina_sn1)

1-element Vector{Num}:
 (-μ) / θ + Ce(t)

### Load remaing subnetwork files

In [10]:
# for i in 1:27
#     include("Retina_SN$(i).jl")
# end
retina_sn2 = include("retina/Retina_SN2.jl")
retina_sn3 = include("retina/Retina_SN3.jl")

Model Retina_SN3:
Unknowns (3): see unknowns(Retina_SN3)
  Sci2(t)
  Ce2(t)
  C2(t)
Parameters (3): see parameters(Retina_SN3)
  β
  κ
  ε

### Find rowspan polynomials of remaining deficiency 1 subnetworks

In [11]:
b, invariant = rowspan_invariants(retina_sn2, [1, 3])
invariant

1×5 Matrix{Num}:
 (-φ) / λ  0  1  0  0

In [12]:
invariant * massactionvector(retina_sn2)

1-element Vector{Num}:
 (-φ) / λ + Ce1(t)

In [13]:
b, invariant = rowspan_invariants(retina_sn3, [1, 3])
invariant

1×5 Matrix{Num}:
 (-β) / κ  0  1  0  0

In [14]:
invariant * massactionvector(retina_sn3)

1-element Vector{Num}:
 (-β) / κ + Ce2(t)

In [15]:
# TODO work out for all subnets
# for sn in subnets
#     complex_ids = [1, 3]
#     _, invariant = rowspan_invariants(sn, complex_ids)
#     display(sn.name)
#     display(invariant)
# end

### Oscar can also be used to find polynomials in both the rowspan and the ideal of the rate equations

In [19]:
f1 = get_oscar_steadystatesystem(retina_sn1)
I1 = ideal(f1)
x = gens(base_ring(I1))
G1 = Oscar.groebner_basis(I1, ordering=lex([x[1],x[3],x[2]]),  complete_reduction = true)

Gröbner basis with elements
  1: k[2]*x[2] - k[1]
  2: k[3]*x[1]*x[3] - k[1]
with respect to the ordering
  lex([x[1], x[3], x[2]])

In [21]:
setpoint_sn1 = (parameters(retina_sn1)[1]) / (parameters(retina_sn1)[2])

μ / θ

In [25]:
f2 = get_oscar_steadystatesystem(retina_sn2)
I2 = ideal(f2)
x = gens(base_ring(I2))
G2 = Oscar.groebner_basis(I2, ordering=lex([x[1],x[3],x[2]]),  complete_reduction = true)

Gröbner basis with elements
  1: k[2]*x[2] - k[1]
  2: k[3]*x[1]*x[3] - k[1]
with respect to the ordering
  lex([x[1], x[3], x[2]])

In [23]:
setpoint_sn2 = (parameters(retina_sn2)[1]) / (parameters(retina_sn2)[2])

φ / λ

In [26]:
f3 = get_oscar_steadystatesystem(retina_sn3)
I3 = ideal(f3)
x = gens(base_ring(I3))
G3 = Oscar.groebner_basis(I3,ordering=lex([x[1],x[3],x[2]]),  complete_reduction = true)

Gröbner basis with elements
  1: k[2]*x[2] - k[1]
  2: k[3]*x[1]*x[3] - k[1]
with respect to the ordering
  lex([x[1], x[3], x[2]])

In [28]:
setpoint_SN3 = (parameters(retina_sn3)[1]) / (parameters(retina_sn3)[2])

β / κ

### Deficiency 3 subnetwork requires Oscar to find RPA polynomials

In [12]:
retina_sn23 = include("retina/Retina_SN23.jl")

Model Retina_SN23:
Unknowns (11): see unknowns(Retina_SN23)
  Cp1(t)
  CO(t)
  Cp(t)
  Ce(t)
  ⋮
Parameters (14): see parameters(Retina_SN23)
  p11
  k5
  p5
  α
  ⋮

In [27]:
parms23 = parameters(retina_sn23)

14-element Vector{Any}:
 p11
 k5
 p5
 α
 ω
 k9
 p12
 k12
 k22
 k24
 k6
 p6
 k21
 p15

### Projecting onto species 1 and 6

In [34]:
f23 = get_oscar_steadystatesystem(retina_sn23)
I23 = ideal(f23)
x = gens(base_ring(I23))
G23 = Oscar.groebner_basis(I23,ordering=lex([x[5],x[2],x[3],x[10],x[11],x[4],x[7],x[8],x[9],x[6],x[1]]),  complete_reduction = true) #should be same information

Gröbner basis with elements
  1: (k[1]*k[12] + k[1]*k[14] + k[3]*k[14])*x[1] - k[5]*k[12]
  2: k[6]*x[6]*x[8] - k[4]
  3: k[7]*x[7]*x[9] - k[5]
  4: (k[1]*k[12]*k[13] + k[1]*k[13]*k[14] + k[3]*k[13]*k[14])*x[4] - k[1]*k[4]*k[12] - k[1]*k[4]*k[14] + 2*k[1]*k[5]*k[12] - k[3]*k[4]*k[14]
  5: (k[1]*k[10]*k[12] + k[1]*k[10]*k[14] + k[3]*k[10]*k[14])*x[11] - k[1]*k[5]*k[12]
  6: (k[1]*k[4]*k[9]*k[12] + k[1]*k[4]*k[9]*k[14] - 2*k[1]*k[5]*k[9]*k[12] + k[3]*k[4]*k[9]*k[14])*x[10] - k[1]*k[5]*k[12]*k[13]
  7: (k[1]*k[2]*k[12]*k[13] + k[1]*k[2]*k[13]*k[14] + k[2]*k[3]*k[13]*k[14])*x[3] - k[1]*k[4]*k[11]*k[12] - k[1]*k[4]*k[11]*k[14] + 2*k[1]*k[5]*k[11]*k[12] + k[1]*k[5]*k[12]*k[13] - k[3]*k[4]*k[11]*k[14]
  8: (k[1]*k[4]*k[8]*k[11]*k[12] + k[1]*k[4]*k[8]*k[11]*k[14] - 2*k[1]*k[5]*k[8]*k[11]*k[12] - k[1]*k[5]*k[8]*k[12]*k[13] + k[3]*k[4]*k[8]*k[11]*k[14])*x[2] - k[1]*k[2]*k[5]*k[12]*k[13]
  9: (k[1]*k[12] + k[1]*k[14] + k[3]*k[14])*x[5] - k[1]*k[5] - k[3]*k[5]
with respect to the ordering
  lex([x

In [30]:
(parms23[5]*parms23[12])/(parms23[1]*parms23[12] + parms23[1]*parms23[14] + parms23[3]*parms23[14])
###FIXME Note this currently doesn't match the setpoint calculated in the paper- need to check this

(p6*ω) / (p11*p15 + p11*p6 + p15*p5)

### Projecting onto species 2 and 6

In [35]:
G23 = Oscar.groebner_basis(I23,ordering=lex([x[5],x[1],x[3],x[10],x[11],x[4],x[7],x[8],x[9],x[6],x[2]]),  complete_reduction = true) #should be same information

Gröbner basis with elements
  1: (k[1]*k[4]*k[8]*k[11]*k[12] + k[1]*k[4]*k[8]*k[11]*k[14] - 2*k[1]*k[5]*k[8]*k[11]*k[12] - k[1]*k[5]*k[8]*k[12]*k[13] + k[3]*k[4]*k[8]*k[11]*k[14])*x[2] - k[1]*k[2]*k[5]*k[12]*k[13]
  2: k[6]*x[6]*x[8] - k[4]
  3: k[7]*x[7]*x[9] - k[5]
  4: (k[1]*k[12]*k[13] + k[1]*k[13]*k[14] + k[3]*k[13]*k[14])*x[4] - k[1]*k[4]*k[12] - k[1]*k[4]*k[14] + 2*k[1]*k[5]*k[12] - k[3]*k[4]*k[14]
  5: (k[1]*k[10]*k[12] + k[1]*k[10]*k[14] + k[3]*k[10]*k[14])*x[11] - k[1]*k[5]*k[12]
  6: (k[1]*k[4]*k[9]*k[12] + k[1]*k[4]*k[9]*k[14] - 2*k[1]*k[5]*k[9]*k[12] + k[3]*k[4]*k[9]*k[14])*x[10] - k[1]*k[5]*k[12]*k[13]
  7: (k[1]*k[2]*k[12]*k[13] + k[1]*k[2]*k[13]*k[14] + k[2]*k[3]*k[13]*k[14])*x[3] - k[1]*k[4]*k[11]*k[12] - k[1]*k[4]*k[11]*k[14] + 2*k[1]*k[5]*k[11]*k[12] + k[1]*k[5]*k[12]*k[13] - k[3]*k[4]*k[11]*k[14]
  8: (k[1]*k[12] + k[1]*k[14] + k[3]*k[14])*x[1] - k[5]*k[12]
  9: (k[1]*k[12] + k[1]*k[14] + k[3]*k[14])*x[5] - k[1]*k[5] - k[3]*k[5]
with respect to the ordering
  lex([x

In [36]:
(parms23[1]*parms23[2]*parms23[5]*parms23[12]*parms23[13]) / (parms23[1]*parms23[4]*parms23[8]*parms23[11]*parms23[12] + parms23[1]*parms23[4]*parms23[8]*parms23[11]*parms23[14] - 2*parms23[1]*parms23[5]*parms23[8]*parms23[11]*parms23[12] - parms23[1]*parms23[5]*parms23[8]*parms23[12]*parms23[13] + parms23[3]*parms23[4]*parms23[8]*parms23[11]*parms23[14])

(k21*k5*p11*p6*ω) / (-k12*k21*p11*p6*ω + k12*k6*p11*p15*α + k12*k6*p11*p6*α - 2k12*k6*p11*p6*ω + k12*k6*p15*p5*α)